In [1]:
import sys; sys.path.append('../')
from antfarm import AntFarm
from antfarm.models import Brownian, Vicsek

In [4]:
AntFarm(Brownian()).start()

Canvas(width=500)

In [3]:
AntFarm(Vicsek()).start()

Canvas(width=500)